# DIGITAL EPIDEMIOLOGY


# Assignment 2 - A study of the Chronic Kidney Disease


# Giacomo Maretto, Gabriel Nespoli

# PART 1: CORRELATION STUDY

# 1.1

Study the correlation between the ground truth data and the Google Trends data. Explore multiple features of the ground truth data (e.g., crude prevalence, age-adjusted prevalence, etc.), and multiple keywords or entity types for querying the Google Trends data. Check for stability (or lack there of) across different years.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Lasso,Ridge
import random
random.seed(123)

In [7]:
# from a dataframe |keyword|correlation|, get a subset of this dataframe according to the threshold for the correlation.
# It will return the keywords with correlation greater or equal than threshold.
def get_most_correlated_terms_df(df, threshold=0.1):
    # order the keywords by the correlation average value
    df.sort_values(by="correlation", ascending=False, inplace=True)
    
    # check if the threshold passed is a float and return a subset of the dataset, otherwise raise an exception
    if isinstance(threshold, float):
        return df.loc[df["correlation"] >= threshold]
    raise AttributeError("Exception in get_most_correlated_terms method. 'threshold' must be float.")

In [8]:
# Load all the dataframes of correlations and get the keywords with the best correlation between the 
# ground truth and google dataset, considering all the years in 'years'
def get_compound_most_correlated_terms_df(file_type="AA", 
                                          years = [2011,2012,2013,2014,2015,2016], threshold=0.1):
    # create an empty dataset that will be filled and returned later
    most_correlated_df = pd.DataFrame(columns=["keyword","correlation"])
    
    # sufix defines the right file to be read, according to the ground-truth data to be analyzed
    sufix = ""
    if file_type == 'AA':
        sufix = "_" + file_type
    
    # iterate over the years populating the aforementioned empty dataframe with the keywords that
    # have a high correlation with the GT data. The resulting dataframe will be a long table
    # with words of each year that attend the condition 'correlation >= threshold'.
    for year in years:
        df = pd.read_csv('overall_correlation_'+ str(year) + sufix + '.csv', sep=';')
        df = get_most_correlated_terms_df(df, threshold=threshold)
        most_correlated_df = most_correlated_df.append(get_most_correlated_terms_df(df, threshold=threshold))
    
    # sort the dataframe by the correlation and drop the repeated keywords, preserving the highest correlated copy
    df.sort_values(by="correlation", ascending=False, inplace=True)
    most_correlated_df.drop_duplicates(subset="keyword", keep="first", inplace=True)
    most_correlated_df.reset_index(drop=True, inplace=True)
    return most_correlated_df

In [9]:
# plot the correlation between the Google Trends keywords and the ground-truth data over the years.
# The y-axis shows the correlation value (between 0 and 1), and the x-axis presents the keywords that
# appear with a high correlation (>=threshold) in all the years. Each line of the graph represents a year.
def plot_keywords_correlation(file_type="AA", years = [2011,2012,2013,2014,2015,2016], threshold=0.1):
    
    # creates an empty dataframe that will be store the values of the plot
    corr_keywords_plot = pd.DataFrame(columns=["keyword", "correlation"])
    years = list(reversed(years))  # the most recent year appears first
    
    # mark the first loop of the iteration. It will define if the corr_keywords_plot will be initialized or,
    # if exists, updated.
    first_execution = True
    
    # create a dataframe with the correlation containing only the terms that repeated in all 'years'. That is,
    # an inner join is done.
    for year in years:
        corr_df = get_compound_most_correlated_terms_df(file_type=file_type, years = [year], threshold=threshold)
        if first_execution:
            corr_keywords_plot = corr_df
            first_execution = False
        else:
            # do an inner join based on the keyword. Only keywords that appear in all the years
            # will be in the plot
            corr_keywords_plot = corr_keywords_plot.merge(corr_df, on=["keyword"], how="inner")
    
    # rename the columns, assigning the year to them
    corr_keywords_plot.columns = ["keyword"] + years
    
    # initialize the plot variables
    fig, ax = plt.subplots()
    
    # plot the curve for each year
    for year in years:
        corr_keywords_plot[year].plot(ax=ax)
    
    # reduce the font size of the legend and move it outside the boxplot
    ax.legend(fontsize="small", bbox_to_anchor=(1, 1))
    
    # define the title and the labels
    if file_type == "AA":
        file_type = "age-adjusted prevalence"
    elif file_type == "CP":
        file_type = "crude prevalence"
    plt.title("Correlation comparison for the " + file_type + " ground-truth data using Google Trends keywords")
    plt.ylabel("Correlation")
    plt.xlabel("Query")
    
    # The plot prints in the x-axis just the position of the keywords (ie. 1,2,3..), not the keywords themselves.
    # A workaround is to map the position to the keyword that it represents
    x_label_sequence = list(range(len(corr_keywords_plot["keyword"].values)))
    plt.xticks(x_label_sequence, corr_keywords_plot["keyword"].values, rotation=90)

In [10]:
plot_keywords_correlation()

The above graph shows the searched terms on Google Trends with highest correlation with the ground-truth data (age-adjusted) in repeated years. This means that only these eight terms had a correlation with the ground-truth higher than 0.1 from 2011 to 2016.

With few counter examples, it is shown that the more recent is the year, the better is the correlation. This can be explained through the growing in the Internet access and usage. This recent increase was specially observed in the older and poorer population. The idea is that the population is more informed and using more Internet to search for symptoms, treatment and prevention of diseases.

Studying deeper the terms, it comes out that they are very significant terms in the universe of the Chronic Kidney Disease, and this affirmation is confirmed by studies, available in the links below:

high blood pressure: https://www.niddk.nih.gov/health-information/kidney-disease/chronic-kidney-disease-ckd/high-blood-pressure

dialysis: https://emedicine.medscape.com/article/238798-treatment

cor pulmonale: https://www.ncbi.nlm.nih.gov/pubmed/22669490

fatty liver kidney: http://care.diabetesjournals.org/content/39/10/1830

gout: https://www.kidney.org/atoz/content/gout

nephroptosis: http://medsideen.bitballoon.com/urinology15/nephroptosis-nephro395.html

renal insuficiency: https://www.ncbi.nlm.nih.gov/pubmed/16344727

In [11]:
plot_keywords_correlation(file_type="CP")

It was found five more terms with significant correlation with the ground-truth data (crude prevalence): diabetes insipidus, renal failure, lupus, diabetic and myotonic muscular dystrophy.

diabetes insipidus: https://www.niddk.nih.gov/health-information/kidney-disease/diabetes-insipidus

renal failure: https://emedicine.medscape.com/article/238798-overview

lupus: https://www.kidney.org/atoz/content/lupus

diabetic: https://www.kidney.org/news/newsroom/factsheets/Diabetes-And-CKD

myotonic muscular dystrophy: https://www.ncbi.nlm.nih.gov/pubmed/29029879

It can be seen the same relation through the years with the ground-truth crude prevalence dataset, except for the keywords gout, nephroptosis, renal insufficiency and myotonic muscular dystrophy.

# 1.2

Study the correlation of your target ground truth feature with income/poverty features.

In [12]:
# define the list of states of the ground truth to be used (it varies according to the year)
df_gt = pd.read_csv('2012_AA.csv', sep=";",usecols=["LocationDesc"])
df_gt.dropna(subset=['LocationDesc'], inplace=True)
df_gt.drop(df_gt.tail(2).index, inplace=True)
states = df_gt["LocationDesc"]

# Calculate the correlation between the average household income of the states and the ground-truth
# data over the years.
def get_income_correlation_df(gt_type="AA", years = [2011,2012,2013,2014,2015,2016]):
    corr = []
    output_df = pd.DataFrame(columns=["year", "correlation"])
    income_df = pd.read_csv("income_per_vector_2011-2016.csv", sep=";")
    
    # gt_type defines the GT that will be used to calculate the correlation with the income
    if gt_type == "AA":
        gt_type = "_AA"
    if gt_type == "CP":
        gt_type = ""
    
    #iterate over the years calculating the correlation of the income with each GT dataset
    for year in years:
        gt_filename = str(year) + gt_type + '.csv'
        gt_df = pd.read_csv(gt_filename, sep=";", usecols=["LocationDesc", "Data_Value"])
        gt_df = gt_df.loc[gt_df["LocationDesc"].isin(states)]  # guarantees that only the 51 states will be used
        corr.append(pearsonr(income_df[str(year)], gt_df["Data_Value"])[0])
    
    # the output is the correlation per year: |year|correlation|
    output_df["year"] = years
    output_df["correlation"] = corr
    output_df.set_index("year", inplace=True)
    
    return output_df

In [13]:
# plot the correlation between the average household income of the states of USA and the ground-truth data
# over the years. The y-axis shows the correlation value (between 0 and 1), and the x-axis presents the years.
# Each line of the graph represents a ground-truth dataset [age-adjusted prevalence, crude prevalence].
def plot_income_correlation(years = [2011,2012,2013,2014,2015,2016]):
    income_df = get_income_correlation_df()
    income_df["Crude Prevalence"] = get_income_correlation_df(gt_type="CP")
    
    # rename the columns of the dataframe
    income_df.columns = ["age-adjusted prevalence", "crude prevalence"]
    
    # plot the curve for each year
    income_df.plot()
    
    # set title and labels of the graph
    plt.title("Correlation comparison of the ground truth data and the average household income")
    plt.ylabel("Correlation")
    plt.xlabel("Year")

In [14]:
plot_income_correlation()

FileNotFoundError: File b'income_per_vector_2011-2016.csv' does not exist

The graph exibit a negative correlation between the average household income of the state in United State and the incidence of the Chronic Kidney Disease. Important factors to sustain this relation is that infectious diseases, poor sanitation, inadequate supply of safe water, environmental pollutants and high concentrations of disease-transmitting vectors  play an important role in the development of CKD in low-income sites. African-Americans, historically a poorer population in USA, can have more than three times more risk to suffer from CKD.

It was also found that there is a circular effect between the poverty and CKD, which means that the patients tends to decrease their income over time.

The aforementioned affirmations are confirmed in the following studies:

https://academic.oup.com/ckj/article/8/1/3/434867
https://www.sciencedirect.com/science/article/pii/S2468024917304795
https://www.kidney.org/news/newsroom/nr/Low-Income-Linked-to-Higher-Levels-of-Kidney-Disease

Moreover, it is noticed that the correlation is higher when the age standardization is applied over the dataset, correcting the discrepancy in the age profile of the population of each state.

# 2.1

In [ ]:
df = pd.read_csv("output_complete.csv",sep=";") # loading dataframe build in previous steps.
df.head()

In [ ]:
train, validation = train_test_split(df, test_size=0.2)

### In the following steps we selected three linear regression models : a simple linear model, Ridge and Lasso.
### For each model we computed the mean squared error.

# Spatial Cross validation
In order to make spatial cross validation we used a sort of Leave-one(state)-out cross validation technique, for each state infact, we splitted the dataframe in train and test set and fitted the models and get a mean.

In [ ]:

#df_gt = pd.read_csv('data/BRFSS/2011_AA.csv', sep=";")
#states = df_gt["LocationDesc"][0:51]


In [ ]:


mseLM = []
mseLasso = []
mseRidge = []
# LOO CrossValidation

trainS=train.drop(["Year"],axis=1)


for state in states:
    trainK = trainS.loc[train["State"] == state]
    testK = trainS.drop(trainK.index)



    #LM
    lm = linear_model.LinearRegression()
    model = lm.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predictions = lm.predict(testK.iloc[:,1:-1])
    mseLM.append(np.mean((predictions - testK.iloc[:,-1])**2))

    #Lasso
    trainKnum = trainK.apply(pd.to_numeric, errors='coerce')
    testKnum = testK.apply(pd.to_numeric, errors='coerce')
    trainKnum.fillna(0, inplace=True)
    testKnum.fillna(0, inplace=True)
    
    lassoReg = Lasso()
    lassoReg.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predLasso = lassoReg.predict(testK.iloc[:,1:-1])
    mseLasso.append(np.mean((predLasso - testK.iloc[:,-1])**2))

    #Ridge
    ridgeReg = Ridge()
    ridgeReg.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predRidge = ridgeReg.predict(testK.iloc[:,1:-1])
    mseRidge.append(np.mean((predRidge - testK.iloc[:,-1])**2))

lmod = np.mean(mseLM)
las = np.mean(mseLasso)
ridge = np.mean(mseRidge)


print("Mean mse LM:",np.mean(mseLM))
print("Mean mse Lasso:",np.mean(mseLasso))
print("Mean mse Ridge:",np.mean(mseRidge))
print("Mean mse in spatial cv: ", np.mean([np.mean(mseLM),np.mean(mseLasso),np.mean(mseRidge)]))

# Temporal cross validation
In order to make a temporal cross validation, we divided in folds the dataset , one for each year and divided in train test set according to this division, computed the models in each fold and get a mean of the error.

In [ ]:
mseLM = []
mseLasso = []
mseRidge = []
trainY=train.drop(["State"],axis=1)


for year in [2011,2012,2013,2014,2015,2016]:
    trainK = trainY.loc[train.Year == year]
    testK = trainY.drop(trainK.index)
    
    
    #lm
    lm = linear_model.LinearRegression()
    model = lm.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predictions = lm.predict(testK.iloc[:,1:-1])
    mseLM.append(np.mean((predictions - testK.iloc[:,-1])**2))


    
    #Lasso
    trainKnum = trainK.apply(pd.to_numeric, errors='coerce')
    testKnum = testK.apply(pd.to_numeric, errors='coerce')
    trainKnum.fillna(0, inplace=True)
    testKnum.fillna(0, inplace=True)
    
    lassoReg = Lasso()
    lassoReg.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predLasso = lassoReg.predict(testK.iloc[:,1:-1])
    mseLasso.append(np.mean((predLasso - testK.iloc[:,-1])**2))

    #Ridge
    ridgeReg = Ridge()
    ridgeReg.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predRidge = ridgeReg.predict(testK.iloc[:,1:-1])
    mseRidge.append(np.mean((predRidge - testK.iloc[:,-1])**2))

lm2 = np.mean(mseLM)
las2 = np.mean(mseLasso)
ridge2 = np.mean(mseRidge)



    

print("Mean mse LM:",np.mean(mseLM))
print("Mean mse Lasso:",np.mean(mseLasso))
print("Mean mse Ridge:",np.mean(mseRidge))
print("Mean mse in temporal cv: ", np.mean([np.mean(mseLM),np.mean(mseLasso),np.mean(mseRidge)]))

## Mean of each model error in the two cross validation types

In [ ]:
print("lm:", np.mean([lmod,lm2]))
print("lasso:", np.mean([las,las2]))
print("ridge:", np.mean([ridge,ridge2]))

## Lasso was the best performing model so we selected it to train on all the train set and test it on the validation set (not used till now)

In [ ]:
# lasso best model
from sklearn.linear_model import Lasso
lassoReg = Lasso(0.2)
lassoReg.fit(train.iloc[:,1:-1],train.iloc[:,-1])
pred2 = lassoReg.predict(validation.iloc[:,1:-1])
# calculating mse
mse = np.mean((pred2 - validation.iloc[:,-1])**2)
print("mse: ",mse)
print("R squared: ",lassoReg.score(validation.iloc[:,1:-1],validation.iloc[:,-1]))

## We let the Lasso linear model select the variables for us, giving them a null coefficients so making their predicting effect absent in the final model.

In [ ]:
predictors = train.iloc[:,1:-1].columns
coef = pd.Series(lassoReg.coef_,predictors).sort_values()
plt.figure(figsize=(20,10))
coef.plot(kind='bar', title='Modal Coefficients',grid=True)
plt.show()
print(lassoReg.coef_)

## Added to our model census features at the US state level (income data)
## Adding the income variable we notice from the mse values that the predicting performance of our lasso model got better.




In [ ]:
income= pd.read_csv("DELIVER/income.csv", sep=";")
_2011i = income.iloc[:,1]
_2012i = income.iloc[:,2]
_2013i = income.iloc[:,3]
_2014i = income.iloc[:,4]
_2015i = income.iloc[:,5]
_2016i = income.iloc[:,6]
income=pd.concat([_2011i,_2012i,_2013i,_2014i,_2015i,_2016i])

In [ ]:
train["income"] = income.iloc[train.index].values
validation["income"] = income.iloc[validation.index].values
#pd.concat([train.income,validation.income],axis=1)

In [ ]:
from sklearn.linear_model import Lasso
train = train.apply(pd.to_numeric, errors='coerce')
validation = validation.apply(pd.to_numeric, errors='coerce')
train.fillna(0, inplace=True)
validation.fillna(0, inplace=True)
lassoReg = Lasso(alpha=0.2)
lassoReg.fit(train.drop(["Data_Value"],axis=1),train.Data_Value)
pred2 = lassoReg.predict(validation.drop(["Data_Value"],axis=1))
# calculating mse

mse = np.mean((pred2 - validation.Data_Value)**2)
print("mse :",mse)
print("R squared: ",lassoReg.score(validation.drop(["Data_Value"],axis=1),validation.Data_Value))

lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
predictions = lm.predict(X_test)
(np.mean((predictions - y_test)**2))



### As a result we get better results (mse reduction from 0.15 to 0.13) adding the "income" variable.

## We also added a state Health Insurance coverage indicator, for all the years

In [ ]:
health = pd.read_csv("DELIVER/health.csv",sep=(";"))

In [ ]:
_2011 = health.iloc[:,1]
_2012 = health.iloc[:,2]
_2013 = health.iloc[:,3]
_2014 = health.iloc[:,4]
_2015 = health.iloc[:,5]
_2016 = health.iloc[:,6]

In [ ]:
health=pd.concat([_2011,_2012,_2013,_2014,_2015,_2016])

In [ ]:
train["health"] = health.iloc[train.index].values
validation["health"] = health.iloc[validation.index].values

In [ ]:

lassoReg1 = Lasso(alpha=0.2)
lassoReg1.fit(train.drop(["Data_Value"],axis=1),train.Data_Value)
pred3 = lassoReg1.predict(validation.drop(["Data_Value"],axis=1))
# calculating mse

mse = np.mean((pred3 - validation.Data_Value)**2)
print("mse :",mse)
print("R squared: ",lassoReg1.score(validation.drop(["Data_Value"],axis=1),validation.Data_Value))

## Strangely we got exact same results of the previous model, so we can see that the model excluded the "health" variable giving a null coefficient, estimating not usefull for an improvement.

In [ ]:
print("Coefficient of health variable:",lassoReg1.coef_[-1])